In [8]:
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [9]:
# Load dataset
df = pd.read_excel('C:/Users/Kornelius/Desktop/Data 2/ecbpr/Warin_Sanger_ECB.xlsx') 
df.head()

,Date,Press,qa
0,1998,"Ladies and gentlemen, in line with our stated ...",We have no indications based on the most recen...
1,1998,"Ladies and gentlemen, as in previous months, t...",The answer to the first question is that it is...
2,1998,"Ladies and gentlemen, the Vice-President and I...",We simply felt that the announcement of a rang...
3,1998,"Ladies and gentlemen, the Vice-President and I...","No, that is not a signal that it is supposed t..."
4,1999,"Ladies and gentlemen, the Vice-President and I...",The main problems have not been technical ones...


In [10]:
#df = df.values.tolist()
# Drop a row by condition
# Print out the first rows of papers
#df = df[df.press]
df = df['Press']

In [11]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
ps = PorterStemmer()
def clean(df):
    stop_free = " ".join([i for i in df.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(ps.stem(word) for word in punc_free.split())
    return normalized

data = [clean(df).split() for df in df]  

In [12]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [13]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(3061 unique tokens: ['abl', 'accept', 'accompani', 'accord', 'account']...)


In [14]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [15]:
# Build the LSI Model
lsi_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=1, decay=0.5)

lsi_model5 = LsiModel(corpus=corpus, id2word=id2word, num_topics=5, decay=0.5)

lsi_model10 = LsiModel(corpus=corpus, id2word=id2word, num_topics=10, decay=0.5)

lsi_model20 = LsiModel(corpus=corpus, id2word=id2word, num_topics=20, decay=0.5)

lsi_model50 = LsiModel(corpus=corpus, id2word=id2word, num_topics=50, decay=0.5)

lsi_model100 = LsiModel(corpus=corpus, id2word=id2word, num_topics=100, decay=0.5)



In [16]:
# View Topics
pprint(lsi_model.print_topics(-1))

# Print the Keyword in the 5 topics
pprint(lsi_model5.print_topics(-1))
doc_lsi = lsi_model5[corpus]

# Print the Keyword in the 5 topics
pprint(lsi_model10.print_topics(-1))
doc_lsi = lsi_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lsi_model20.print_topics(-1))
doc_lsi = lsi_model20[corpus]

# Print the Keyword in the 20 topics
pprint(lsi_model50.print_topics(-1))
doc_lsi = lsi_model50[corpus]

# Print the Keyword in the 40 topics
pprint(lsi_model100.print_topics(-1))
doc_lsi = lsi_model100[corpus]

[(0,
  '0.280*"price" + 0.275*"euro" + 0.267*"growth" + 0.258*"area" + 0.225*"rate" '
  '+ 0.197*"monetari" + 0.197*"econom" + 0.171*"remain" + 0.159*"inflat" + '
  '0.150*"continu"')]
[(0,
  '0.280*"price" + 0.275*"euro" + 0.267*"growth" + 0.258*"area" + 0.225*"rate" '
  '+ 0.197*"monetari" + 0.197*"econom" + 0.171*"remain" + 0.159*"inflat" + '
  '0.150*"continu"'),
 (1,
  '-0.328*"council" + -0.276*"govern" + -0.217*"euro" + -0.204*"ecb" + '
  '0.202*"inflat" + 0.192*"remain" + 0.166*"expect" + 0.147*"medium" + '
  '-0.127*"central" + -0.122*"monetari"'),
 (2,
  '-0.435*"price" + -0.240*"stabil" + -0.228*"growth" + 0.177*"loan" + '
  '-0.153*"risk" + 0.146*"measur" + 0.146*"polici" + 0.143*"expect" + '
  '-0.143*"strong" + -0.139*"develop"'),
 (3,
  '0.232*"expect" + 0.221*"govern" + -0.219*"monetari" + -0.219*"polici" + '
  '0.170*"council" + -0.165*"area" + 0.165*"medium" + 0.162*"remain" + '
  '0.145*"financi" + 0.141*"pressur"'),
 (4,
  '0.408*"euro" + 0.367*"area" + -0.229*"rate

 (9,
  '0.224*"pressur" + -0.200*"credit" + -0.167*"medium" + -0.162*"subdu" + '
  '0.158*"measur" + 0.158*"recent" + 0.153*"project" + -0.149*"condit" + '
  '-0.144*"improv" + -0.133*"rate"'),
 (10,
  '0.345*"project" + -0.280*"growth" + 0.179*"econom" + -0.171*"moder" + '
  '0.161*"staff" + -0.146*"purchas" + -0.135*"increas" + 0.134*"ecb" + '
  '0.131*"activ" + -0.124*"loan"'),
 (11,
  '-0.348*"price" + 0.240*"market" + 0.182*"financi" + 0.149*"growth" + '
  '-0.126*"increas" + 0.121*"favour" + -0.120*"inflat" + -0.119*"energi" + '
  '0.117*"measur" + -0.114*"gradual"'),
 (12,
  '0.240*"euro" + -0.228*"growth" + 0.210*"area" + -0.200*"market" + '
  '-0.184*"financi" + -0.175*"recent" + 0.174*"stabil" + 0.140*"measur" + '
  '0.133*"support" + 0.130*"ahead"'),
 (13,
  '0.229*"develop" + 0.218*"increas" + 0.159*"price" + -0.153*"level" + '
  '-0.149*"bank" + -0.149*"risk" + 0.148*"eurosystem" + -0.143*"interest" + '
  '-0.133*"rate" + -0.128*"escb"'),
 (14,
  '0.293*"continu" + -0.187*

  '-0.139*"april" + 0.122*"regard" + 0.108*"activ" + -0.104*"like" + '
  '0.104*"two" + -0.103*"broadli" + 0.102*"howev" + 0.099*"upsid" + '
  '-0.099*"sinc" + -0.099*"march"'),
 (71,
  '-0.132*"strengthen" + 0.130*"sector" + -0.126*"unchang" + -0.125*"first" + '
  '-0.124*"confid" + 0.122*"effect" + -0.118*"financi" + 0.112*"today" + '
  '0.107*"current" + 0.106*"releas"'),
 (72,
  '0.153*"overal" + -0.149*"current" + 0.141*"contribut" + 0.127*"second" + '
  '-0.122*"possibl" + -0.122*"may" + 0.117*"continu" + 0.113*"march" + '
  '0.102*"effect" + -0.101*"juli"'),
 (73,
  '-0.166*"may" + -0.162*"continu" + 0.148*"contribut" + 0.137*"bank" + '
  '-0.111*"higher" + 0.110*"increas" + 0.107*"declin" + 0.104*"balanc" + '
  '0.102*"due" + -0.101*"import"'),
 (74,
  '0.127*"remain" + 0.121*"improv" + -0.119*"develop" + -0.116*"last" + '
  '-0.115*"still" + 0.114*"concern" + -0.114*"therefor" + -0.110*"high" + '
  '0.103*"may" + -0.103*"privat"'),
 (75,
  '0.183*"improv" + -0.145*"energi" + 0

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [17]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Perplexity not working?
#print('\nPerplexity: ', lsi_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



# Compute Coherence Score
coherence_model_lsi5 = CoherenceModel(model=lsi_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi5 = coherence_model_lsi5.get_coherence()
print('\nCoherence Score5: ', coherence_lsi5)

# Compute Coherence Score
coherence_model_lsi10 = CoherenceModel(model=lsi_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi10 = coherence_model_lsi10.get_coherence()
print('\nCoherence Score10: ', coherence_lsi10)

# Compute Coherence Score
coherence_model_lsi20 = CoherenceModel(model=lsi_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi20 = coherence_model_lsi20.get_coherence()
print('\nCoherence Score20: ', coherence_lsi20)

# Compute Coherence Score
coherence_model_lsi50 = CoherenceModel(model=lsi_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi50 = coherence_model_lsi50.get_coherence()
print('\nCoherence Score50: ', coherence_lsi50)

# Compute Coherence Score
coherence_model_lsi100 = CoherenceModel(model=lsi_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi100 = coherence_model_lsi100.get_coherence()
print('\nCoherence Score100: ', coherence_lsi100)


# Compute Coherence Score
coherence_model_lsi5 = CoherenceModel(model=lsi_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi5 = coherence_model_lsi5.get_coherence()
print('\nCoherence Score5: ', coherence_lsi5)

# Compute Coherence Score
coherence_model_lsi10 = CoherenceModel(model=lsi_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi10 = coherence_model_lsi10.get_coherence()
print('\nCoherence Score10: ', coherence_lsi10)

# Compute Coherence Score
coherence_model_lsi20 = CoherenceModel(model=lsi_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi20 = coherence_model_lsi20.get_coherence()
print('\nCoherence Score20: ', coherence_lsi20)

# Compute Coherence Score
coherence_model_lsi50 = CoherenceModel(model=lsi_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi50 = coherence_model_lsi50.get_coherence()
print('\nCoherence Score50: ', coherence_lsi50)

# Compute Coherence Score
coherence_model_lsi100 = CoherenceModel(model=lsi_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi100 = coherence_model_lsi100.get_coherence()
print('\nCoherence Score100: ', coherence_lsi100)


Coherence Score5:  0.3533440026791765

Coherence Score10:  0.3743689304492044

Coherence Score20:  0.32947374420228737

Coherence Score50:  0.27297817912499583

Coherence Score100:  0.24415444492446914

Coherence Score5:  -0.20784169510161016

Coherence Score10:  -0.31962468868931504

Coherence Score20:  -0.3409058808932573

Coherence Score50:  -0.7759786357855151

Coherence Score100:  -0.6392028431224204
